<a href="https://colab.research.google.com/github/FranciscoOcampoPredictiva/azureml_course/blob/main/Lecture_1_Automated_ML_in_Azure_Machine_Learning_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Installation and Setup

In [ ]:
# Install azureml SDK package
! pip install -q azureml-sdk

In [ ]:
# Importing the class
from azureml.core import Workspace

In [ ]:
# Accessing the workspace and creating a workspace object
ws = Workspace.from_config(path='/content/config.json')

# Step 2 - Accessing the input data

In [ ]:
input_ds = ws.datasets.get('Churn Modelling Data SDK')

# Step 3 - Creating the compute cluster

In [ ]:
cluster_name = 'azureml-sdk-cluster'

# Import the class AmlCompute
from azureml.core.compute import AmlCompute

# Provision the configuration using AmlCompute
if cluster_name not in ws.compute_targets:
  compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D11_V2',
                                                         max_nodes=2)
  cluster = AmlCompute.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
  cluster.wait_for_completion()
else:
  cluster = ws.compute_targets[cluster_name]

# Step 4 - Configuring the AutoML run

In [ ]:
# Import the class AutoMLConfig
from azureml.train.automl import AutoMLConfig

# Create an object of the class AutoMLConfig
automl_config = AutoMLConfig(task='classification',
                             compute_target=cluster,
                             training_data=input_ds,
                             validation_size=0.3,
                             label_column_name='Exited',
                             primary_metric='accuracy',
                             iterations=10,
                             max_concurrent_iterations=2,
                             experiment_timeout_hours=0.5,
                             featurization='auto')

# Step 5 - Creating and submitting the experiment run

In [ ]:
# Import the class Experiment
from azureml.core.experiment import Experiment

# Create the experiment
new_exp = Experiment(workspace=ws, name='automl-experiment')

# Submit the experiment run
new_run = new_exp.submit(automl_config)
new_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***********************************************************************

{'runId': 'AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347',
 'target': 'azureml-sdk-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-18T03:10:40.196846Z',
 'endTimeUtc': '2022-09-18T03:20:37.982953Z',
 'services': {},
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0.3',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'azureml-sdk-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"76d4f7e1-3c6a-41a3-968f-043a0cb83503","resource_group":"azureml-sdk-rg","workspace_name":"azureml-sdk-ws","region":"westus","compute_target":"azureml-sdk-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_

# Step 6 - Getting the best model

In [ ]:
new_run.get_best_child(metric='accuracy')

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_8,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


# Step 7 - Getting the metrics for all the runs

In [ ]:
# Get the metrics for all the runs
for run in new_run.get_children():
    print("")
    print("Run ID : ", run.id)
    print(run.get_metrics('accuracy'))


Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_8
{'accuracy': 0.8673333333333333}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_9
{'accuracy': 0.8656666666666667}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_7
{'accuracy': 0.832}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_6
{'accuracy': 0.8466666666666667}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_5
{'accuracy': 0.7986666666666666}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_4
{'accuracy': 0.865}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_worker_1
{}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_3
{'accuracy': 0.8633333333333333}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_2
{'accuracy': 0.6926666666666667}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_1
{'accuracy': 0.8573333333333333}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_0
{'accuracy': 0.8626666666666667}

Run ID :  AutoML_62f9970b-54fe-4eb9-afec-60b8c5dcb347_work